<a href="https://colab.research.google.com/github/eiliscasey17/QuantumMadness/blob/main/Qantum_Madness_Dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd

### NOTE: Make sure you run all cells when editing + re-running code

In [29]:
matchups = pd.read_csv("Tournament Matchups.csv")
print(matchups.head())
shooting_splits = pd.read_csv("Shooting Splits.csv")
shooting_splits.head()

   YEAR  BY YEAR NO  BY ROUND NO  TEAM NO           TEAM  SEED  ROUND  \
0  2025        2162         2166     1141         Auburn     1      1   
1  2025        2161         2165     1145    Alabama St.    16     64   
2  2025        2160         2164     1141         Auburn     1      1   
3  2025        2159         2163     1099  Saint Francis    16     64   
4  2025        2158         2162     1119     Louisville     8      1   

   CURRENT ROUND  SCORE  
0             64    NaN  
1             64    NaN  
2             64    NaN  
3             64    NaN  
4             64    NaN  


,YEAR,TEAM NO,TEAM ID,TEAM,CONF,DUNKS FG%,DUNKS SHARE,DUNKS FG%D,DUNKS D SHARE,CLOSE TWOS FG%,...,CLOSE TWOS FG%D RANK,CLOSE TWOS D SHARE RANK,FARTHER TWOS FG% RANK,FARTHER TWOS SHARE RANK,FARTHER TWOS FG%D RANK,FARTHER TWOS D SHARE RANK,THREES FG% RANK,THREES SHARE RANK,THREES FG%D RANK,THREES D SHARE RANK
0,2025,1147,2,Akron,MAC,85.9,7.0,80.0,3.7,61.8,...,24,66,22,56,61,8,20,14,28,26
1,2025,1146,3,Alabama,SEC,90.4,13.7,88.7,7.9,62.4,...,35,30,1,66,53,60,34,12,13,9
2,2025,1145,4,Alabama St.,SWAC,77.0,5.2,81.0,5.6,53.1,...,41,50,67,39,41,21,56,20,49,36
3,2025,1144,6,American,Pat,76.2,2.2,94.9,3.6,58.5,...,62,58,37,37,43,36,32,16,52,13
4,2025,1143,8,Arizona,B12,95.3,9.5,85.7,6.9,65.9,...,28,6,46,19,47,54,60,57,52,48


In [30]:
filtered = matchups.loc[matchups['YEAR'] < 2025, ['YEAR', 'TEAM NO', 'TEAM', 'SEED', 'CURRENT ROUND', 'SCORE']]

team1 = filtered.iloc[::2].reset_index(drop=True)  # Rows 0,2,4,...
team2 = filtered.iloc[1::2].reset_index(drop=True)  # Rows 1,3,5,...

#combine datasets to create one row for each game
combined = pd.concat([team1.add_suffix('1'), team2.add_suffix('2')], axis=1)


In [31]:
#select columns from shooting splits i want
# shooting_splits = shooting_splits[[ 'TEAM NO',
#     'DUNKS FG%', 'DUNKS SHARE', 'DUNKS FG%D', 'DUNKS D SHARE',
#     'CLOSE TWOS FG%', 'CLOSE TWOS SHARE', 'CLOSE TWOS FG%D', 'CLOSE TWOS D SHARE',
#     'FARTHER TWOS FG%', 'FARTHER TWOS SHARE', 'FARTHER TWOS FG%D', 'FARTHER TWOS D SHARE',
#     'THREES FG%', 'THREES SHARE', 'THREES FG%D', 'THREES D SHARE',
#     'DUNKS FG% RANK', 'DUNKS SHARE RANK', 'DUNKS FG%D RANK', 'DUNKS D SHARE RANK',
#     'CLOSE TWOS FG% RANK', 'CLOSE TWOS SHARE RANK', 'CLOSE TWOS FG%D RANK', 'CLOSE TWOS D SHARE RANK',
#     'FARTHER TWOS FG% RANK', 'FARTHER TWOS SHARE RANK', 'FARTHER TWOS FG%D RANK', 'FARTHER TWOS D SHARE RANK',
#     'THREES FG% RANK', 'THREES SHARE RANK', 'THREES FG%D RANK', 'THREES D SHARE RANK'
# ]]

shooting_splits = shooting_splits[[ 'TEAM NO',
    'DUNKS FG%' ]]

# Merge shooting splits for TEAM1
combined = combined.merge(shooting_splits.add_suffix('1'), left_on='TEAM NO1', right_on='TEAM NO1', how='left')

# Merge shooting splits for TEAM2
combined = combined.merge(shooting_splits.add_suffix('2'), left_on='TEAM NO2', right_on='TEAM NO2', how='left')

#set winning team each row
combined.loc[combined['SCORE1'] > combined['SCORE2'], 'WINNER'] = 0
combined.loc[combined['SCORE1'] < combined['SCORE2'], 'WINNER'] = 1

#drop redundant columns
combined.drop('CURRENT ROUND2', axis=1, inplace=True)
combined.drop('YEAR2', axis=1, inplace=True)

combined.rename(columns={'YEAR1': 'YEAR', 'CURRENT ROUND1': 'CURRENT ROUND'}, inplace=True)

combined.head()

,YEAR,TEAM NO1,TEAM1,SEED1,CURRENT ROUND,SCORE1,TEAM NO2,TEAM2,SEED2,SCORE2,DUNKS FG%1,DUNKS FG%2,WINNER
0,2024,1067,Connecticut,1,64,91.0,1026,Stetson,16,52.0,92.1,91.0,0.0
1,2024,1060,Florida Atlantic,8,64,65.0,1036,Northwestern,9,77.0,82.9,86.3,1.0
2,2024,1029,San Diego St.,5,64,69.0,1020,UAB,12,65.0,83.3,88.5,0.0
3,2024,1076,Auburn,4,64,76.0,1012,Yale,13,78.0,87.7,93.9,1.0
4,2024,1073,BYU,6,64,67.0,1062,Duquesne,11,71.0,88.9,77.9,1.0
